# PerfCounters Usage


This notebook showcases how to use the [PerfCounters package](https://github.com/ebursztein/perfcounters).

let's start by importing the PerfCounters class which is all you need:

In [53]:
import time
import random
import numpy as np
from perfcounters import PerfCounters
from IPython.core.display import HTML

## Basic usage

### Time measurement usage
Short example that uses the timing counters to compare `random()` versus `randint()` to get a value in  {0, 1}.

In [23]:
counters = PerfCounters()  # init the counter collection.

counters.start('random')  # start a timing counter called random.random.
for x in range(100000):
    int(random.random())
counters.stop('random')  # stop the random.random.

counters.start('randint')  # start a timing counter called random.random.
for x in range(1000000):
    random.randint(0,1)
counters.stop('randint')  # stop the random.random.

counters.report() # report print all counter values in nicely formated tables.

-=[Timing counters]=-
+---------+-----------+
| name    |     value |
|---------+-----------|
| randint | 0.982396  |
| random  | 0.0229394 |
+---------+-----------+




You can also export the results in various formats including json, text, grepable text and HTML.
For example to export in json you can simply use the `to_json()` function as follow: 

In [5]:
counters.to_json()

'{"Timing counters": [["exponentiation", 0.010947942733764648]]}'

### Laps measurement usage

Timing counters are used to track how long each iteration of a loop is taking. They
work as follow:

In [33]:
counters = PerfCounters()  # declaring our counters

counters.start('random loop') # create counter
for _ in range(3):
    time.sleep(round(random.random(), 2))
    counters.lap('random loop') # record lap time
counters.stop('random loop')

counters.report()  # we don't need to stop the counter. Report do it

-=[Timing counters]=-
+-------------+---------+
| name        |   value |
|-------------+---------|
| random loop | 1.36072 |
+-------------+---------+

-=[Laps counters]=-

-= random loop =-
+------------+-------------------+
|   lap time |   cumulative time |
|------------+-------------------|
|   0.640529 |          0.640529 |
|   0.520168 |          1.1607   |
|   0.200028 |          1.36072  |
+------------+-------------------+
+---------+----------+
| stat    |    value |
|---------+----------|
| min     | 0.200028 |
| average | 0.453575 |
| median  | 0.520168 |
| max     | 0.640529 |
| stddev  | 0.185896 |
+---------+----------+



As visible in the output above when outputing/returning laps counters `PerfCounters` do report the value of each lap, the cumulative time and statistics about the laps.

### Value counters usage
Value counters used to track values. They are either directly set to
a given value with the `set()` method  or incremented with the `increment()` method.


In [36]:
counters = PerfCounters()
counters.set('mycounter', 39)  # set counter value to 39
counters.increment('mycounter', 3)  # increment counter by 3
val = counters.get('mycounter') #  get the value of the counter
print('mycounter value:', val)

mycounter value: 42


## Complete example
Here is an end to end example that demonstrate all the basic feature of `PerfCounters` at once.

In [43]:
from perfcounters import PerfCounters
from random import randint

# init counters
counters = PerfCounters()

num_iterations = randint(5, 10)

# setting a value counter to a given value
counters.set('num_iterations', num_iterations)

# starting a timing counter
counters.start('loop')

for i in range(num_iterations):
    for _ in range(randint(1000, 50000)):
        v = randint(0, 1)

    # incrementing a value counter to sum the generated values
    counters.increment('total_value', v)

    # track lap time
    counters.lap('loop')

# stopping the timing counter
counters.stop('loop')

# reporting all counters
counters.report()

-=[Value counters]=-
+----------------+---------+
| name           |   value |
|----------------+---------|
| num_iterations |       6 |
| total_value    |       2 |
+----------------+---------+

-=[Timing counters]=-
+--------+----------+
| name   |    value |
|--------+----------|
| loop   | 0.167572 |
+--------+----------+

-=[Laps counters]=-

-= loop =-
+------------+-------------------+
|   lap time |   cumulative time |
|------------+-------------------|
| 0.00798035 |        0.00798035 |
| 0.0189776  |        0.026958   |
| 0.0468643  |        0.0738223  |
| 0.0289271  |        0.102749   |
| 0.0339088  |        0.136658   |
| 0.0309143  |        0.167572   |
+------------+-------------------+
+---------+------------+
| stat    |      value |
|---------+------------|
| min     | 0.00798035 |
| average | 0.0279287  |
| median  | 0.0299207  |
| max     | 0.0468643  |
| stddev  | 0.0121263  |
+---------+------------+



# Using deadline warning
Here is how to use `PerfCounters` to emit a log warning if a counter exceed a specific deadline. This is for example useful when using remote service and wanting to track when they take too long.

In [44]:
counters = PerfCounters()
# counter will emit a log warning if time between start and stop exceed 1sec.
counters.start('deadline_monitor', warning_deadline=1)
time.sleep(2)
counters.stop('deadline_monitor')

counter deadline_monitor deadline exceeded. Operation took:                               2.000716209411621 secs. Deadline was: 1 secs


## Sorting
`report()` and any export function such as to_html() and to_text() support counters sorting as follow

In [47]:
counters = PerfCounters()
counters.set('a', 42)
counters.set('b', 40)
counters.set('c', 41)

print("sort by value desc (default)")
counters.report() # or counters.report(sort_by='value', reverse=True)

print("sort by value asc")
counters.report(reverse=False)  # or counters.report(sort_by='value', reverse=False)

print("sort by name asc")
counters.report(sort_by='name', reverse=False)

print("sort by name desc")
counters.report(sort_by='name')

sort by value desc (default)
-=[Value counters]=-
+--------+---------+
| name   |   value |
|--------+---------|
| a      |      42 |
| c      |      41 |
| b      |      40 |
+--------+---------+


sort by value asc
-=[Value counters]=-
+--------+---------+
| name   |   value |
|--------+---------|
| b      |      40 |
| c      |      41 |
| a      |      42 |
+--------+---------+


sort by name asc
-=[Value counters]=-
+--------+---------+
| name   |   value |
|--------+---------|
| a      |      42 |
| b      |      40 |
| c      |      41 |
+--------+---------+


sort by name desc
-=[Value counters]=-
+--------+---------+
| name   |   value |
|--------+---------|
| c      |      41 |
| b      |      40 |
| a      |      42 |
+--------+---------+




## Export functions

`PerfCounters` support various export formats:

In [62]:
counters = PerfCounters()
counters.start('loop')
for i in range(1000):
    v = randint(0, 1000000)
    counters.increment('total_value', v)
counters.stop('loop')

### JSON


In [61]:
print(counters.to_json())

{"Timing counters": [["loop", 0.0019681453704833984]], "Value counters": [["total_value", 500036794]]}


### HTML

In [60]:
HTML(counters.to_html())

name,value
total_value,500036794
name,value
loop,0.00196815


### text


In [63]:
print(counters.to_text())

-=[Value counters]=-
+-------------+-----------+
| name        |     value |
|-------------+-----------|
| total_value | 515219551 |
+-------------+-----------+

-=[Timing counters]=-
+--------+------------+
| name   |      value |
|--------+------------|
| loop   | 0.00199723 |
+--------+------------+




### Grepable text

In [65]:
print(counters.to_grepable_text())

[PerfCounters]Value counters:total_value:515219551
[PerfCounters]Timing counters:loop:0.001997232437133789



## Counters merging

`PerfCounters` support scope and merging so you can combine counters from various part of your code. 
Here is a basic example that showcase how to use this feature

In [67]:
counters = PerfCounters()
counters.set('test', 42)

# set counter prefix via constructor to avoid name collision
other_counters = PerfCounters('others')
other_counters.set('test', 42)

# merging
counters.merge(other_counters)
counters.report()


-=[Value counters]=-
+-------------+---------+
| name        |   value |
|-------------+---------|
| test        |      42 |
| others_test |      42 |
+-------------+---------+


